1.В исходном датасете мультииндексов нет, поэтому будем создавать их сами. Данные сохранены в переменную pokemon.
  Сначала измените названия исходных столбцов:
    - пробелы и точки нужно заменить на "_" (напр. Sp. Atk --> sp_atk).
    - приведите все названия к нижнему регистру
    - колонку "#" переименовать в "id"

Полученные результаты запишите в исходный датафрейм pokemon.   
Затем сгруппируйте данные по поколению покемонов (generation),   
и с помощью value_counts() посчитайте, сколько в каком поколении легендарных покемонов (legendary).   
Полученный объект приведите к формату датафрейма (.to_frame()) и сохраните в legends.   
Данные сохранены в переменную pokemon.   

In [3]:
import pandas as pd
url = 'https://stepik.org/media/attachments/lesson/363874/Pokemon.csv'
pokemon = pd.read_csv(url)

In [4]:
pokemon = pokemon.rename(columns={'#':'id', 'Name':'name', 
                                  'Type 1':'type_1', 'Type 2':'type_2',
                                  'Total':'total', 'HP': 'hp',
                                  'Attack':'attack', 'Defense':'defense',
                                  'Sp. Atk': 'sp_atk', 'Sp. Def': 'sp_def',
                                  'Speed': 'speed', 'Generation': 'generation',
                                  'Legendary': 'legendary'})
legends = pokemon.groupby('generation').legendary.value_counts().to_frame()


In [5]:
legends

legendary
generation legendary           
1          False            160
           True               6
2          False            101
           True               5
3          False            142
           True              18
4          False            108
           True              13
5          False            150
           True              15
6          False             74
           True               8

2. Как вы могли заметить,  есть только одна колонка со значениями, которая называется legendary, а в качестве индекса используются две колонки — generation и legendary. Два одинаковых названия — не очень хорошо, поэтому необходимо переименовать колонку legendary в legendary_count. 

    - Используйте датафрейм legends, полученный на предыдущем шаге, и измените в нём название столбца, перезаписав его в ту же переменную. 
    - Затем используйте unstack, чтобы поместить уровень индекса legendary в уровень оси столбцов. Иными словами, должно получиться две колонки – False & True. Результат сохраните в legends_unstacked.

In [6]:
legends = legends.rename({'legendary' : 'legendary_count'}, axis = 1)
legends_unstacked = legends.unstack()
legends_unstacked

legendary_count      
legendary            False True 
generation                      
1                      160     6
2                      101     5
3                      142    18
4                      108    13
5                      150    15
6                       74     8

3. Немного усложним задачу. Теперь попробуем узнать, среди каких типов покемонов и какого поколения больше всего легендарных.

Сгруппируйте датасет pokemon по переменным generation и type_1, посчитайте количество легендарных покемонов внутри групп. Приведите данные в формат датафрейма, а затем используйте unstack(). В качестве ответа выберите вид и поколение покемона, среди которых больше всего легендарных.

Hint: обратиться к колонке с мультииндексом можно с помощью .loc[:,('legendary', True)]

In [7]:
legends = pokemon.query('legendary == True')
legends = legends.groupby(['generation', 'type_1'])
#legends.query('legendary == True').legendary.value_counts().to_frame()
legends.legendary.value_counts().to_frame().idxmax()

legendary    (3, Dragon, True)
dtype: object

4. Преобразуйте представленные данные в длинный формат и запишите в переменную avocado_agg_long. В качестве индекса используйте type
~~~
avocado_agg = pd.DataFrame({'type' : ['conventional', 'organic'],
                            'AvgPrice_2015' : [1.077963, 1.673324],
                            'AvgPrice_2016' : [1.105595, 1.571684],
                            'AvgPrice_2017' : [1.294888, 1.735521],
                            'AvgPrice_2018' : [1.127886, 1.567176],
                            })
~~~

In [8]:
avocado_agg = pd.DataFrame({'type' : ['conventional', 'organic'],
                            'AvgPrice_2015' : [1.077963, 1.673324],
                            'AvgPrice_2016' : [1.105595, 1.571684],
                            'AvgPrice_2017' : [1.294888, 1.735521],
                            'AvgPrice_2018' : [1.127886, 1.567176],
                            })
avocado_agg

,type,AvgPrice_2015,AvgPrice_2016,AvgPrice_2017,AvgPrice_2018
0,conventional,1.077963,1.105595,1.294888,1.127886
1,organic,1.673324,1.571684,1.735521,1.567176


In [9]:
avocado_agg_long = pd.wide_to_long(avocado_agg, ['AvgPrice'], i=['type'], j='year', sep='_')
avocado_agg_long

,,AvgPrice
type,year,
conventional,2015,1.077963
organic,2015,1.673324
conventional,2016,1.105595
organic,2016,1.571684
conventional,2017,1.294888
organic,2017,1.735521
conventional,2018,1.127886
organic,2018,1.567176


5.  Имеется набор данных о супергероях в широком формате. В первой колонке Name находятся их имена, а остальные 167 столбцов — различные характеристики (суперсилы), принимающие значение либо True, либо False. Давайте преобразуем датафрейм так, чтобы "собрать" эти признаки в один столбец под названием superpower, а в качестве значений поместить туда списки имеющихся у того или иного героя суперсил. 

То есть привести в следующий формат
~~~

           Name                                         superpower
568  Spider-Man  [Reflexes, Animal Oriented Powers, Danger Sens...
~~~
Сначала приведите данные к длинному формату, где единственным идентификатором будет имя героя Name. Новый столбец с названиями суперсил переименуйте в superpower. Полученный датафрейм запишите в superheroes_long.

Для того, чтобы соединить силы в списки и привести данные в желаемую форму, отфильтруйте колонку value так, чтобы остались строки только со значением True. Сгруппируйте датасет по Name, после чего возьмите столбец superpower и используйте .apply(list). Результат сохраните в superheroes_powers. Обратите внимание, что результирующий объект тоже должен быть датафреймом, а колонки должно быть всего две — Name и superpower.

Данные записаны в superheroes.

In [12]:
url = 'https://stepik.org/media/attachments/lesson/363874/superheroes_power_matrix.csv'
superheroes = pd.read_csv(url)

In [13]:
superheroes_long = pd.melt(superheroes, id_vars = 'Name')
superheroes_long = superheroes_long.rename(columns =  {'variable' : 'superpower'})
superheroes_powers = superheroes_long.query('value == True').groupby('Name').superpower.apply(list).reset_index()
superheroes_powers

,Name,superpower
0,3-D Man,"[Agility, Super Strength, Stamina, Super Speed]"
1,A-Bomb,"[Accelerated Healing, Durability, Longevity, S..."
2,Abe Sapien,"[Agility, Accelerated Healing, Cold Resistance..."
3,Abin Sur,[Lantern Power Ring]
4,Abomination,"[Accelerated Healing, Intelligence, Super Stre..."
...,...,...
662,Yellowjacket II,"[Flight, Energy Blasts, Size Changing]"
663,Ymir,"[Cold Resistance, Durability, Longevity, Super..."
664,Yoda,"[Agility, Stealth, Danger Sense, Marksmanship,..."
665,Zatanna,"[Cryokinesis, Telepathy, Magic, Fire Control, ..."
